In [1]:
from pathlib import Path
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

In [2]:
def load_pose_data(path: Path):
    data = []
    with open(path, "r") as f:
        for line in f:
            coordinates = []
            for coord_str in line.strip().split():
                x, y, z = map(float, coord_str.split(","))
                coordinates.append([x, y, z])
            data.append(coordinates)

    matrix = np.array(data)
    return matrix

In [3]:
data_path = Path('data/pose')
pose_file_paths = list(data_path.glob('*.dat'))

In [5]:
m = load_pose_data(pose_file_paths[0])

In [6]:
m.shape

(2000, 31, 3)

In [10]:
m

array([[[ 0.        ,  0.        ,  0.        ],
        [ 0.00469088, -0.3213831 ,  0.08153991],
        [ 0.00846129, -0.57670665,  0.15275946],
        ...,
        [ 0.10168944, -1.3145258 ,  0.13505237],
        [ 0.1512783 , -1.2166059 ,  0.34355485],
        [ 0.01010621, -1.3203782 ,  0.10308437]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.00381304, -0.31957448,  0.0883164 ],
        [ 0.00691916, -0.57632536,  0.15416402],
        ...,
        [ 0.10368826, -1.3191913 ,  0.14315774],
        [ 0.1512797 , -1.2124177 ,  0.347723  ],
        [ 0.0123293 , -1.3272802 ,  0.11105645]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.00380774, -0.31957236,  0.08832388],
        [ 0.00690962, -0.57632333,  0.15417099],
        ...,
        [ 0.10349564, -1.3192029 ,  0.14302391],
        [ 0.1511991 , -1.2125461 ,  0.34762478],
        [ 0.01212451, -1.3272216 ,  0.11094096]],

       ...,

       [[ 0.        ,  0.        ,  0.        ],
        [ 0

In [13]:
m[:10].shape

(10, 31, 3)